# Protein Language Models Fundamentals

## Introduction

**Learning Objectives**

- What is encoding?
- What is a embedding?
- Downloading a model from HuggingFace
- How to get feature embeddings for your sequence

In this section we will run through the fundamental steps and components in a **Protein Language Model**, starting with encoding our protein sequence into a form a computer can recognise to converting it to a *vector* that contains its information about the sequence and its individual amino acids.

## Encoding

Previously we generated several mutated seqeunces given our data. These are represented as a string of single, captital letters. This method of encoding is good for humans, but for computers the sequence needs to be converted into numbers.

As typical with machine learning this numerical representation will be *tensor* of values where each **vector** will denote which amino acid is in each position.

KEY TERMS BOX

**Vector**: A sequence of numbers that represents a point in a multi-dimensional space.

Each protein sequence is composed of **20 common types** of amino acids. Therefore, one of the simplest methods to encode a protein sequence is through **one-hot encoding**. In this encoding scheme, each amino acid is represented by a binary vector of length 20. Each position in the vector corresponds to one of the 20 amino acids, and the position representing the specific amino acid in the sequence is set to 1, while all other positions are set to 0. This method effectively transforms the categorical amino acid data into a numerical format that can be used in machine learning models.

For example, we have a protein sequence: ACDET, and we would like to identify 'A' as 0, 'C' as 1, 'D' as 2, and so forth up to 'T' as 19 as the following pattern:

A one-hot encoding for a protein sequence containing the amino acids A, C, D, E, and T might be represented as follows:

| Amino Acid | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 |
|------------|---|---|---|---|---|---|---|---|---|---|----|----|----|----|----|----|----|----|----|----|
| A          | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| C          | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| D          | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| E          | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
| T          | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 1  |

## Embedding

One-hot encoding is a straightforward way to show what kind of amino acids are in a protein, but it's a bit like just knowing someone's name without knowing anything more about them. Proteins are more complex than that. To really understand what a protein can do, we need to know more than just the amino acid types; we need to understand how these amino acids interact, how they've evolved, and what physical and chemical properties they have. But gathering all this detailed information for every amino acid in every protein can be overwhelming and often not feasible.

To make things simpler and more informative, we use **embeddings**. Imagine if you could create a small summary or a 'mini-profile' for each amino acid that includes not just its name but also hints at its deeper characteristics—like its behavior, its role in the protein, and its history. That's what an embedding does. It turns each amino acid into a compact set of numbers (this set is what we call a *vector*). You can decide how many numbers there should be, but each set uniquely reflects crucial aspects of the amino acid. For example, if you have 100 sequences, and assume each protein is so complex that each of the amino acid should be represented by 256 features, your can identify your sequences as a tensor with the shape of `[100, 256]`.

TIP BOX

You can find more information about embedding via this link: [LLM Embeddings](https://huggingface.co/spaces/hesamation/primer-llm-embedding?section=what_are_embeddings?).

### How to get embedding vectors

Protein language models like ESM work in two stages to create meaningful representations of amino acids:

**Stage 1: Initial Embeddings**  
During training on 250 million protein sequences, the model learns a unique numerical representation (embedding vector) for each of the 20 amino acids. These embeddings are stored in the model's **embedding layer**. This is the 'mini-profile' that we mentioned earlier, a numerical capture of the general and historical properties of an amino acid type.

**Stage 2: Contextualisation through Attention**  
However, the same amino acid can play very different roles depending on its context in the sequence. This is where the **attention mechanism** comes in. After the initial embeddings are assigned, the model passes them through multiple attention layers (33 layers in ESM-1b). Each layer refines the embeddings by considering the surrounding amino acids. The result is that an alanine at position 50 will have a different final embedding depending on whether it's surrounded by hydrophobic residues in a protein core or charged residues on the surface.

The resulting contextualised embeddings from protein language models have been proven to implicitly capture structural and evolutionary characteristics of proteins. This means that even without explicitly programming features like secondary structure or conservation scores into the model, the trained embeddings naturally incorporate this crucial biological information through learning from millions of sequences.

By utilising the embeddings generated from these pretrained protein language models, we can apply them to various downstream prediction tasks without needing to retrain on massive datasets ourselves. 

For instance, [VariPred](https://github.com/wlin16/VariPred) employs embeddings from the **ESM model** to train a missense mutation pathogenicity predictor. A workflow we will be replicating from here on.

**Language models** are not just about predicting the next work or amino acid in a sequence, they are also a rich, pre-encoded insights embedded within a high-dimensional *vector*.

With that in mind, let's extract **contextualised embeddings** from our prepared protein sequences using one of the most powerful protein language models, **ESM-1b**.

## Loading the Model

Most protein language models with transformer architecture are stored on [HuggingFace](https://huggingface.co/).  

We mentioned it previously at the end of [**AI2**](../AI2/pre-trained-software.html#downloading-pre-trained-modles), however here is another brief synopsis:

KEY TERMS BOX

HuggingFace is a platform that centralises various pre-trained models, including protein language models with transformer architecture. It simplifies the process of downloading and training these models, making it easier for researchers and developers to access state-of-the-art models for natural language processing tasks. This platform is important because it promotes reproducibility, standardisation, and collaboration in the field of machine learning. By providing a centralised hub for models, HuggingFace streamlines the workflow of experimenting with different models and accelerates the development of new applications.

### Setup

To use transformer based models from **huggingface** you will need to install the [**transformers** library](https://pypi.org/project/transformers/), which provides API access to the models. To find the models you want, head to the huggingface website and search for for your use case. Clicking on a model will take you to a page where you can find the model card, which will contain information about the model, including the model's name, description, and the code need to load it. For example, see [here](https://huggingface.co/facebook/esm1b_t33_650M_UR50S) for the **ESM1b model**.

In [ ]:
import torch
from transformers import EsmModel, AutoTokenizer

In [ ]:
SEED = 42
torch.manual_seed(SEED)
# Set the seed for all devices (both CPU and CUDA)
torch.cuda.manual_seed_all(SEED)

# Check if GPU is available
if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device(f"cuda:0")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'Device name: {torch.cuda.get_device_name(0)}')
else:
    print("No GPU detected! Falling back to CPU")
    # If the GPU is not available, use the CPU
    device = torch.device("cpu")

In [ ]:
# Load the ESM model with its pretrained weights
model = EsmModel.from_pretrained("facebook/esm1b_t33_650M_UR50S", use_safetensors=True)
model.to(device)

# Load the tokenizer corresponding to the ESM model
tokenizer = AutoTokenizer.from_pretrained("facebook/esm1b_t33_650M_UR50S")

From the **transformers** library we have imported the **ESM** model and a tokeniser,  **AutoTokenizer**. We will use the tokeniser to encode our text data into a format that the model can understand.

TIP BOX

Don't worry about the warning stating some weights are missing. This is because the EsmModel purposely has some downstream layers untrained for the user to train for their own task. In our case we will just be using the model to generate embeddings and then creating our own prediction neural net.

### Checking the contents of a PyTorch model

Let's check what's inside the **ESM1b** model:

In [ ]:
print(model)

The model's architecture is complex and intricate. However, don't worry. You do not need to know the specifics of each layer. 

What's crucial to understand is that this model consists of 33 identical **transformer** blocks. The final output layer has a dimension of **1280**, indicating that the model will represent each amino acid as a vector with **1280 scalar values**. This vector representation is commonly referred to as an *embedding*.

### Model parameters

Having a look at the total model trainig parameters, we can see that this is a fairly large model with just over **650 million parameters** for training. It is this volume of parameters that will make obtain the embeddings so time consuming at the end of this section.

In [ ]:
# The trainable parameters
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)

print(f'{total_trainable_params:,} training parameters.')

## Tokenisation

Tokenisation is a crucial step in natural language processing tasks. It involves breaking down text into smaller units, such as words or subwords, to facilitate further processing by machine learning models. In the context of protein sequences, tokenisation is essential to convert the raw amino acid sequences into a format that can be understood and processed by the model. By tokenising protein sequences, we enable the model to learn patterns and relationships within the sequence data, ultimately improving its performance in downstream tasks.

NOTE BOX

Tokenisation is different to embedding. The tokens are the **keys** (numerical) that the model knows to be associated with its pre-trained embeddings. 

In [ ]:
# Tokenizer can convert protein sequences into a list of index tokens
seq = 'ACFGPLOIE'
print(seq)
print(f"Sequence of length: {len(seq)}", "\n")

tokens = tokenizer(seq, return_tensors="pt")
print(tokens['input_ids'])
print(f"Tokens of length: {len(tokens['input_ids'][0])}")

The tokeniser converts a protein sequence into index tokens according to a vocabulary dictionary (see below).

However, you will notice that the token tensor is 2 items longer than the input sequence. 
This is because the tokeniser adds a special token `[cls]` at the beginning of the sequence and a `[eos]` (indicates "end of sequence") at the end of the sequence.
You can either keep or ignore the these special tokens. But you need to be aware of them.

To have a better understanding about the index showing above in the 'input_ids', you can view the token dictionary by using .get_vocab() function

In [ ]:
token_dict = tokenizer.get_vocab()

for c, (k, v) in enumerate(token_dict.items()):
    print(f"{k}: {v}")

    if c == 8:
        break

PRACTICE EXERCISE

Convert the sequence `AOPIEYTLKQGFMCV` into tokens.

## Fetch an embedding

After converting the sequence into tokens, we can use the model to get the **embeddings** of the tokenised sequence.

In [ ]:
seq = 'AOPIEYTLKQGFMCV'
new_seq_token = tokenizer(seq,return_tensors="pt")
new_seq_token.to(device)

with torch.no_grad():
    outputs = model(**new_seq_token) # generate the embeddings

outputs.keys()

The `outputs` contains two keys: `last_hidden_state` and `pooler_output`. `last_hidden_state` is the sequence embedding from the last layer of the model. Usually, we take `last_hidden_state` as the representation of the input protein sequence.  

In [ ]:
outputs['last_hidden_state'].shape

### Breaking down the tensor `[1, 17, 1280]`

- The first dimension indicates there is only `one` sequence. If we feed two sequences into the model, the first dimension will be two.
- The second dimension refers to the length of the protein sequence. Note, as mentioned above, the input_token includes two special tokens `[cls]` and `[eos]`. Thus, though the length of the original sequence is 15, the length of the final input fed into the model is 17
- The third dimension here is the `fixed dimension` of representation. Here, the dimension is 1280, which indicates that each amino acid is represented by 1280 scalars.

### Embeddings are context dependent:

Ass disussed in the prior sub--section the **embedding vector** for each amino acid will change dependent on where it is in a sequence and what sequence it is in due to the **attention mechanism**, in other words it is **context independent**. 

This can be seen by comparing the amino acid `'A'` (Alanine) when on its own or part of `'AO'` (Alanine-Pyrrolysine):

In [ ]:
# Define our sequences and tokenise
seqA = 'A'
seqA_token = tokenizer(seqA,return_tensors="pt")
seqA_token.to(device)

seqAO = 'AO'
seqAO_token = tokenizer(seqAO,return_tensors="pt")
seqAO_token.to(device)

# Get embeddings
with torch.no_grad():
    seqA_output = model(**seqA_token)
    seqAO_output = model(**seqAO_token)

# Extract the embedding of only Alanine
a_embedding_own = seqA_output['last_hidden_state'][0][1] 
a_embedding_withO = seqAO_output['last_hidden_state'][0][1]

print(f"Embedding of A (on its own): {a_embedding_own}")
print(f"Embedding of A (on its own): {a_embedding_withO}")
print(" ")
print(f"Are they equivalent: {all(a_embedding_own == a_embedding_withO)}")

### Multiple sequences:

To fetch embeddings for multiple sequences, you can simply put the sequences as individual items in a list:

In [ ]:
seq = ['AOPIEYTLKQGFMCV','MOPDEYTWK']
new_seq_token = tokenizer(seq, return_tensors="pt")

However, in most cases, the sequences in the input list have different lengths, thus we will get embeddings of different sizes. As a tensor needs to be a matrix, our embeddings need to be the same length.

To achieve this we use padding. Padding is a process where we add a special token to the end of the sequence to make it the same length as the longest sequence in the list. 

In this case we add the argument `True` to the `padding` parameter. Here, the `[pad]` token (which is 1 in this case) to the end of the shorter sequences. Additionally, it will use "0" in the attention_mask to indicate which tokens are `[pad]` tokens. This ensures that all sequences are padded to the same length, enabling the model to process them in batches effectively.

In [ ]:
seq = ['AOPIEYTLKQGFMCV','MOPDEYTWK']
new_seq_tokens = tokenizer(seq, return_tensors="pt", padding=True)
# To move all the token sequences to device in one go, use dictionary comprehension
new_seq_tokens = {k: v.to(device) for k, v in new_seq_tokens.items()}
with torch.no_grad():
    outputs = model(**new_seq_tokens) # generate the embeddings

outputs['last_hidden_state'].shape

When we give a list of **two** sequences and pad them, the value in the first dimension increases by **one** to **two**.

## Embedding Extraction

We will generate embeddings using **ESM-1b** for both the wildtype sequence and the mutated sequence.

However, ESM-1b can only accept protein sequence with the maximum length of **1022**, some of our seqeunces are longer than this. 

In [ ]:
import pandas as pd

# Load in our processed data
train = pd.read_csv("./data/processed_train.csv")
test = pd.read_csv("./data/processed_test.csv")

print(f"Number of sequences > 1022; Train: {len(train[train['Length'] > 1022])}, Test: {len(test[test['Length'] > 1022])}")

There are multiple ways to process the protein sequences, to ensure they fit our *model*. The [**VariPred**](https://github.com/wlin16/VariPred) package applies a **truncation** stratergy around the mutation point. Here, for the sake of demonstration purposes we will remove the sequences that are too long. 

In [ ]:

train = train[train['Length'] <= 1022].copy()
test = test[test['Length'] <= 1022].copy()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (8, 4))

ax.hist(train["wt_seq"].str.len(), alpha = 0.5, density=True, label = "Train")
ax.hist(test["wt_seq"].str.len(), alpha = 0.5, density=True, label = "Test")
ax.set_title("Distribution of Sequence Lengths (density)")
ax.legend()

plt.show()

Now, finally, our dataset is ready to be used by the model. 

### Generate embeddings from the model

There are two ways to deal with the feature extraction during the classification model training:

1. Generate the embeddings for the given dataset repeatly during the training.
2. Generate the embeddings from the dataset once, save it as a binary file into your disk. Read such embedding file into the RAM (memory) when we need to train the model.

The first method saves disk storage space, however, it usually takes a much longer time because the embeddings of each sequence have to be generated again and again during the training.

The second method needs to occupy a bit more storage space as it stores the generated embeddings to the disk. However, since we can directly feed the embedding into the model for training, the entire training process can speed up dramatically.

In this case, we will apply the second method.

## Embeddings to disk

The previous histograms show that the sequence lengths varies a lot even after we trunctated the sequences. Embeddings can be generated for these sequences at the same time by calling the model on the list of sequences (as we demonstrated above). However for the amound of sequences we would like to produce, this will consume a lot of memory dur to parallel computing and padding.

### Why?

Let's image we need apply the tokeniser for mixed sequences with the shortest length (54) and the longest length (1022). The input tokens fed into the model will always be padded into the length of the longest sequence (1022). Generating the embedding for a sequence with only 54 amino acid will only take: $$54 * 1280 * 4 / 1024 /1024 = 0.27GB$$, but this number will increase to about 5GB if we are going to generate the embedding for a sequence with 1022 amino acids.

So when encountering amino acid sequences with different length it can be more efficient to implement your own tokeniser/embedding workflow with a `for-loop`.

WARNING BOX

Typically trying to run the original workflow will crash the workspace environment due to using up all avaiable RAM. Try it for yourself and see:

```
seq_test = test["wt_seq"].tolist()

seq_tokens = tokenizer(seq_test, return_tensors="pt", padding=True)
seq_tokens = {k: v.to(device) for k, v in seq_tokens.items()} # Move tokeniser output to device 

with torch.no_grad():
    outputs = model(**seq_tokens) # generate the embeddings
```

This will cause the kernel to crash.


### Implementation

For the next workflow we will generate the **embeddings** for the amino acids occuring at the mutation poistion, for both the *wildtype* and *mutated* sequences. Together, they will be a representation for the **wt-mt** sequence pair.

To obtain this we will need to generate the whole embedding for each sequence, to ensure its *context dependent*, and then select just the embeddings at our known mutational point. These two embedding can then be *concatenated* together to provide a vector with information from both.

In [ ]:
from tqdm import tqdm

def get_embeddings(dataset):
    
    # Initialise an empty dictionary to store the data
    data_dict = {"target_id": [], "embeddings": [], "labels": []}
    
    # Iterate over the rows in the test dataset
    with torch.no_grad():

        for index, row in tqdm(dataset.iterrows()):  # Use index along with row

            # Tokenise the wildtype and mutated sequences
            wt_token = tokenizer(row['wt_seq'], return_tensors="pt")
            wt_token = {k: v.to(device) for k, v in wt_token.items()} # Move tokeniser output to device 
            
            mt_token = tokenizer(row['mt_seq'], return_tensors="pt")
            mt_token = {k: v.to(device) for k, v in mt_token.items()} # Move tokeniser output to device 

            # Get the index of the amino acid
            aa_index = row['aa_index']

            # Generate embeddings for wildtype and mutated sequences
            wt_emb = model(**wt_token)
            mt_emb = model(**mt_token)
            
            # Extract the last_hidden_state (the actual embeddings) from the model output
            wt_hidden = wt_emb['last_hidden_state'] 
            mt_hidden = mt_emb['last_hidden_state'] 
            
            # Extract the embeddings at the specified amino acid index
            wt_out = wt_hidden[:, aa_index, :].cpu().detach()
            mt_out = mt_hidden[:, aa_index, :].cpu().detach()
            
            # Concatenate the wildtype and mutated amino acids' embeddings
            concat = torch.cat([wt_out, mt_out], dim=-1).squeeze(0)  # Concatenate along feature dimension
            
            # Store the id, concatenated embeddings and label in the dictionary
            data_dict['target_id'].append(row['target_id'])
            data_dict['embeddings'].append(concat)
            data_dict['labels'].append(row['label'])
    
    return data_dict


To save the data for the next section we can store the resultant data dictionary as a pickle file. 

The [pickle module](https://docs.python.org/3/library/pickle.html) converts any Python object into a *byte stream* than can be later decoded back into the same object when loaded. It is one of the go to methods for saving in Python and is one of the *built-in* modules that doesn't require installing.

In [ ]:
import pickle

Running the above function for both the training and test dataset would take a prohibitively long time on just a **CPU**. Running it on a local computer was calculated to take roughly **25 hours** for the *train* dataset (80 000 samples).

Even when running it with a **GPU** the process still took approximately **5 hours** for the *train* dataset. Given this we have stored the embeddings in the `./data` folder in your repository.

The code below would be what you would need to run.

In [ ]:
train_data_dict = get_embeddings(train)

with open('./data/embs_for_train.pkl', 'wb') as file:
    pickle.dump(train_data_dict, file)
    print('Data saved successfully')

In [ ]:
test_data_dict = get_embeddings(test)

with open('./data/embs_for_test.pkl', 'wb') as file:
    pickle.dump(test_data_dict, file)
    print('Data saved successfully')

## Summary

In this section we have progressed from mutational sequences to a **scaler vector** that is encoded with historical, contextual information about both the *wildtype* and *mutated* amino acid. 

In the next section, we are going to train a classifier on the representations that have been generated